## **Exempel: Multi-AI-agenter för bokning av hotell**

I dagens snabbrörliga värld innebär planering av en affärsresa mer än bara att boka en flygresa och ett hotellrum. Det krävs en nivå av samordning och effektivitet som kan vara svår att uppnå. Det är här Multi-AI-agenter kommer in i bilden, och revolutionerar sättet vi hanterar våra resebehov på.

Föreställ dig att ha ett team av intelligenta agenter till ditt förfogande, som tillsammans sköter varje aspekt av din resa med precision och lätthet. Med vår avancerade AI-teknologi har vi skapat specialiserade agenter för bokningstjänster och resplanering, vilket garanterar en sömlös och stressfri reseupplevelse.

Det här är ett grundläggande scenario. Vid planering av en affärsresa behöver vi rådgöra med en affärsreseagent för att få information om flygbiljetter, hotellinformation osv. Genom AI-agenter kan vi skapa agenter för bokningstjänster och agenter för resplanering som samarbetar för att förbättra intelligensnivån.


# Initiera Azure AI Agent-tjänsten och hämta konfigurationsinformation från **.env**

### **.env**

Skapa en .env-fil

**.env** innehåller anslutningssträngen för Azure AI Agent-tjänsten, modellen som används av AOAI och motsvarande Google API-sökningstjänsts API, ENDPOINT, etc.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Ditt namn på modellutplacering för Azure AI Agent-tjänsten"

[**NOTE**] Du behöver en modell med 100 000 Rate Limit (Tokens per minut) och Rate Limit på 600 (Förfrågningar per minut)

  Du kan få modellen i Microsoft Foundry - Model and Endpoint.


- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Din anslutningssträng för projektet i Azure AI Agent-tjänsten"

  Du kan få anslutningssträngen för projektet i din projektöversikt i AI Foundry Portal-skärmen.

- **SERPAPI_SEARCH_API_KEY** = "Din SERPAPI Search API NYCKEL"
- **SERPAPI_SEARCH_ENDPOINT** = "Din SERPAPI Search Endpoint"

För att få Model Deployment Name och Project Connection String för Azure AI Agent-tjänsten, behöver du skapa Azure AI Agent-tjänsten. Det rekommenderas att använda [denna mall](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) för att skapa den direkt （***Note:*** Azure AI Agent-tjänsten är för närvarande inställd i en begränsad region. Det rekommenderas att du hänvisar till [denna länk](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) för att ställa in regionen)

Agenten behöver åtkomst till SERPAPI. Det rekommenderas att registrera sig med [denna länk](https://serpapi.com/searches). Efter registrering kan du erhålla en unik API NYCKEL och ENDPOINT


# Logga in på Azure 

Du behöver nu logga in på Azure Öppna en terminal i VScode och kör kommandot `az login`


# Setup 

To run this notebook, you will need to install the following libraries. Here is a list of the required libraries and the corresponding pip install commands:

azure-identity: För Azure-autentisering.
requests: För att göra HTTP-förfrågningar.
semantic-kernel: För ramverket semantic kernel (förutsatt att detta är ett anpassat eller specifikt bibliotek, kan det vara nödvändigt att installera det från en specifik källa eller repository).


In [ ]:
!pip install azure-identity
!pip install requests
!pip install semantic-kernel
!pip install --upgrade semantic_kernel
!pip install azure-cli

# Förklaring: 
import asyncio: Detta importerar asyncio-modulen, som ger stöd för asynkron programmering i Python. Det gör det möjligt att skriva samtidiga kod med async- och await-syntaxen.
from typing import Annotated: Detta importerar typen Annotated från typing-modulen. Annotated används för att lägga till metadata till typanvisningar, vilket kan vara användbart för olika ändamål såsom validering, dokumentation eller verktyg.


In [ ]:
import asyncio,os
from typing import Annotated

# Förklaring:
Genom att använda from dotenv import load_dotenv och load_dotenv(), kan du enkelt hantera konfigurationsinställningar och känslig information (som API-nycklar och databas-URL:er) i en .env-fil, hålla dem separata från din källkod och göra din applikation mer säker och lättare att konfigurera.


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Förklaring:

Import-sats: from azure.identity.aio import DefaultAzureCredential: Detta importerar klassen DefaultAzureCredential från modulen azure.identity.aio. aio-delen av modulnamnet indikerar att det är utformat för asynkrona operationer.

Syftet med DefaultAzureCredential: Klassen DefaultAzureCredential är en del av Azure SDK för Python. Den tillhandahåller ett standard sätt att autentisera sig mot Azure-tjänster. Den försöker autentisera med flera metoder i en specifik ordning, såsom miljövariabler, hanterad identitet och Azure CLI-referenser.

Asynkrona operationer: aio-modulen indikerar att klassen DefaultAzureCredential stödjer asynkrona operationer. Detta innebär att du kan använda den med asyncio för att utföra icke-blockerande autentiseringsförfrågningar.


In [ ]:
from azure.identity.aio import DefaultAzureCredential

# Förklaring:
Importerar olika moduler och klasser från paketet semantic_kernel. Här är en översikt av varje import:

AgentGroupChat från semantic_kernel.agents: Denna klass hanterar funktionaliteter relaterade till gruppchatt för AI-agenter. AzureAIAgent och AzureAIAgentSettings från semantic_kernel.agents.azure_ai

AzureAIAgent: Denna klass används för att skapa och hantera AI-agenter som använder Azure AI-tjänster.

AzureAIAgentSettings: Denna klass används för att konfigurera inställningar för AzureAIAgent. TerminationStrategy från semantic_kernel.agents.strategies.termination.termination_strategy:

Denna klass definierar strategier för att avsluta exekveringen av AI-agenter under vissa förhållanden. ChatMessageContent från semantic_kernel.contents.chat_message_content:

Denna klass används för att hantera innehållet i chattmeddelanden.
AuthorRole från semantic_kernel.contents.utils.author_role:

Denna klass definierar olika roller för författare i sammanhanget av chattmeddelanden.

kernel_function från semantic_kernel.functions.kernel_function_decorator: Denna dekoratör används för att definiera kernel-funktioner, vilka är funktioner som kan exekveras inom semantic kernel-ramverket.
Dessa importer sätter upp de nödvändiga komponenterna för att skapa och hantera AI-agenter som kan interagera i en gruppchattmiljö, eventuellt för uppgifter som att boka hotell eller liknande aktiviteter.


In [ ]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.contents import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function

# Förklaring:
Nästa importerar vi CodeInterpreterTool-klassen från modulen azure.ai.projects.models.

CodeInterpreterTool: Denna klass är en del av Azure AI SDK och används för att tolka och köra kod inom ramen för AI-projekt. Den tillhandahåller funktioner för att köra kodstycken, analysera kod eller integrera kodkörning inom AI-arbetsflöden.
Denna import förbereder den nödvändiga komponenten för att använda CodeInterpreterTool i ditt projekt, vilket kan vara användbart för uppgifter som innebär att tolka och köra kod dynamiskt.


In [ ]:
from azure.ai.projects.models import CodeInterpreterTool

# Förklaring: 
ApprovalTerminationStrategy-klassen ger en specifik strategi för att avsluta en AI-agents verksamhet. Agenten kommer att avsluta om det senaste meddelandet i dess interaktionshistorik innehåller ordet "saved". Detta kan vara användbart i scenarier där agentens uppgift anses vara slutförd när den får en bekräftelse på att något har "saved". Definiera interaktionsmetoden. Efter att reservationsplanen har sparats kan den stoppas när sparsignalen mottas.


In [ ]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "saved" in history[-1].content.lower()

# Förklaring:

Kodraden initierar ett AzureAIAgentSettings-objekt med standard- eller fördefinierade inställningar genom att anropa create()-metoden. Detta inställningsobjekt (ai_agent_settings) kan sedan användas för att konfigurera och hantera en AzureAIAgent-instans.


In [ ]:
ai_agent_settings = AzureAIAgentSettings.create()

# Förklaring:
Genom att importera requests-biblioteket kan du enkelt göra HTTP-förfrågningar och interagera med webb-tjänster i din Python-kod.


In [ ]:
import requests

# Förklaring:
Detta är en variabel som lagrar API-nyckeln för att få tillgång till en SERP (Search Engine Results Page) API-tjänst. En API-nyckel är en unik identifierare som används för att autentisera förfrågningar kopplade till ditt konto.

'GOOGLE_SEARCH_API_KEY': Detta är en platshållarsträng. Du behöver ersätta ''GOOGLE_SEARCH_API_KEY' med din faktiska SERP API-nyckel.

Syfte: Syftet med denna rad är att lagra API-nyckeln i en variabel så att den kan användas för att autentisera förfrågningar till SERP API-tjänsten. API-nyckeln krävs för att få tillgång till tjänsten och utföra sökningar.

Hur man får en SERP API-nyckel: För att få en SERP API-nyckel, följ dessa allmänna steg på https://serpapi.com (de exakta stegen kan variera beroende på den specifika SERP API-tjänst du använder):

Välj en SERP API-tjänst: Det finns flera SERP API-tjänster tillgängliga, såsom SerpAPI, Google Custom Search JSON API och andra. Välj den som bäst passar dina behov.

Registrera ett konto:

Gå till webbplatsen för den valda SERP API-tjänsten https://www.serpapi.com och registrera ett konto. Du kan behöva ange viss grundläggande information och verifiera din e-postadress.

Skapa en API-nyckel:

Efter registrering loggar du in på ditt konto och navigerar till API-avsnittet eller instrumentpanelen. Leta efter ett alternativ för att skapa eller generera en ny API-nyckel.
Kopiera API-nyckeln:

När API-nyckeln har genererats, kopiera den. Denna nyckel kommer att användas för att autentisera dina förfrågningar till SERP API-tjänsten.
Ersätt platshållaren:

Ersätt platshållaren i din .env-fil


In [ ]:
SERPAPI_SEARCH_API_KEY=os.getenv('SERPAPI_SEARCH_API_KEY')

In [ ]:
SERPAPI_SEARCH_ENDPOINT = os.getenv('SERPAPI_SEARCH_ENDPOINT')

# Förklaring:
BookingPlugin-klassen tillhandahåller metoder för att boka hotell och flyg med hjälp av Serpapi.com Google Search API. Den konstruerar nödvändiga parametrar, skickar API-förfrågningar och bearbetar svaren för att returnera relevant bokningsinformation. API-nyckeln (SERPAPI_SEARCH_API_KEY) och slutpunkten (SERPAPI_SEARCH_ENDPOINT) används för att autentisera och skicka förfrågningar till Google Search API.


In [ ]:
# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""
    @kernel_function(description="booking hotel")
    def booking_hotel(self,query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-in Time"])-> Annotated[str, "Return the result of booking hotel infomation"]:

        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "2",
            "currency": "USD",
            "gl": "us",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY
        }

        response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=params)
        if response.status_code == 200:
            response = response.json()
            return response["properties"]
        else:
            return None

    
    @kernel_function(description="booking fight")
    def  booking_fight(self,origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"])-> Annotated[str, "Return the result of booking fight infomation"]:
        
        go_params = {
            "engine": "google_flights",   
            "departure_id": origin,
            "arrival_id": destination,
            "outbound_date": outbound_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }

        print(go_params)

        go_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=go_params)


        result = ''

        if go_response.status_code == 200:
            response = go_response.json()

            result += "# outbound \n " + str(response)
        else:
            print('error!!!')
            # return None

        
        back_params = {
            "engine": "google_flights",   
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": return_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }


        print(back_params)


        back_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=back_params)



        if back_response.status_code == 200:
            response = back_response.json()

            result += "\n # return \n"  + str(response)

        else:
            print('error!!!')
            # return None
        
        print(result)

        return result

        


# Förklaring:
SavePlugin-klassen tillhandahåller en metod saving_plan för att spara reseplaner med hjälp av Azure AI-tjänster. Den ställer in Azure-referenser, skapar en AI-agent, bearbetar användarens indata för att generera och spara reseplanens innehåll samt hanterar filsparande och städningsoperationer. Metoden returnerar "Saved" vid lyckad slutförande.


In [ ]:
class SavePlugin:
    """Save Plugin for customers"""
    @kernel_function(description="saving plan")
    async def saving_plan(self,tripplan: Annotated[str, "The content of trip plan"])-> Annotated[str, "Return status of save content"]:

        async with (
            DefaultAzureCredential() as creds,
            AzureAIAgent.create_client(
                credential=creds,
                conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
            ) as client,
        ):

            code_interpreter = CodeInterpreterTool()
            
            agent_definition = await client.agents.create_agent(
                model=ai_agent_settings.model_deployment_name,
                tools=code_interpreter.definitions,
                tool_resources=code_interpreter.resources,
            )


            agent = AzureAIAgent(
                client=client,
                definition=agent_definition,
            )

            thread = await client.agents.create_thread()


            user_inputs = [
                """
            
                        You are my Python programming assistant. Generate code,save """+ tripplan +
                        
                    """    
                        and execute it according to the following requirements

                        1. Save blog content to trip-{YYMMDDHHMMSS}.md

                        2. give me the download this file link
                    """
            ]



            try:
                for user_input in user_inputs:
                    # Add the user input as a chat message
                    await agent.add_chat_message(
                        thread_id=thread.id, message=ChatMessageContent(role=AuthorRole.USER, content=user_input)
                    )
                    print(f"# User: '{user_input}'")
                    # Invoke the agent for the specified thread
                    async for content in agent.invoke(thread_id=thread.id):
                        if content.role != AuthorRole.TOOL:
                            print(f"# Agent: {content.content}")

                    
                    messages = await client.agents.list_messages(thread_id=thread.id)

                    # OpenAIPageableListOfThreadMessage
                    # OpenAIPageableListOfThreadMessage


                    for file_path_annotation in messages.file_path_annotations:

                            file_name = os.path.basename(file_path_annotation.text)

                            await client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./trip")

                    
            finally:
                await client.agents.delete_thread(thread.id)
                await client.agents.delete_agent(agent.id)


        return "Saved"

# Förklaring:
Den här koden sätter upp Azure AI-agenter för att hantera bokning av flyg och hotell samt spara reseplaner baserat på användarens indata. Den använder Azure-uppgifter för att skapa och konfigurera agenterna, bearbetar användarens indata genom en gruppchatt och säkerställer korrekt städning efter att uppgifterna är slutförda. Agenterna använder specifika plugin-program (BookingPlugin och SavePlugin) för att utföra sina respektive uppgifter.


In [ ]:
async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):
    BOOKING_AGENT_NAME = "BookingAgent"
    BOOKING_AGENT_INSTRUCTIONS = """
    You are a booking agent. Help me book flights or hotels.

    Thought: Please understand the user's intention and confirm whether to use the reservation system to complete the task.

    Actions:
    - For flight bookings, convert the departure and destination names into airport codes.
    - Use the appropriate API for hotel or flight bookings. Verify that all necessary parameters are available. If any parameters are missing, ask the user to provide them. If all parameters are complete, call the corresponding function.
    - If the task is not related to hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
      - For flight bookings, output separate outbound and return contents in the order of:
        Departure Airport | Airline | Flight Number | Departure Time | Arrival Airport | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
      - For hotel bookings, output in the order of:
        Property Name | Property Description | Check-in Time | Check-out Time | Prices | Nearby Places | Hotel Class | GPS Coordinates.
    """

    SAVE_AGENT_NAME = "SaveAgent"
    SAVE_AGENT_INSTRUCTIONS = """
    You are a save tool agent. Help me to save the trip plan.
    """

    # Create agent definition
    booking_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=BOOKING_AGENT_NAME,
        instructions=BOOKING_AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent
    booking_agent = AzureAIAgent(
        client=client,
        definition=booking_agent_definition,
        # Optionally configure polling options
        # polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
    )

    # Add the sample plugin to the kernel
    booking_agent.kernel.add_plugin(BookingPlugin(), plugin_name="booking")

    # Create agent definition
    save_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=SAVE_AGENT_NAME,
        instructions=SAVE_AGENT_INSTRUCTIONS
    )

    # Create the AzureAI Agent
    save_agent = AzureAIAgent(
        client=client,
        definition=save_agent_definition,
    )

    save_agent.kernel.add_plugin(SavePlugin(), plugin_name="saving")

    user_inputs = [
        "I have a business trip from London to New York in Feb 20 2025 to Feb 27 2025 ,help me to book a hotel and fight tickets and save it"
    ]

    chat = AgentGroupChat(
        agents=[booking_agent, save_agent],
        termination_strategy=ApprovalTerminationStrategy(agents=[save_agent], maximum_iterations=10),
    )

    try:
        for user_input in user_inputs:
            # Add the user input as a chat message
            await chat.add_chat_message(
                ChatMessageContent(role=AuthorRole.USER, content=user_input)
            )
            print(f"# User: '{user_input}'")

            async for content in chat.invoke():
                print(f"# {content.role} - {content.name or '*'}: '{content.content}'")

            print(f"# IS COMPLETE: {chat.is_complete}")

            print("*" * 60)
            print("Chat History (In Descending Order):\n")
            async for message in chat.get_chat_messages(agent=save_agent):
                print(f"# {message.role} - {message.name or '*'}: '{message.content}'")
    finally:
        await chat.reset()
        await client.agents.delete_agent(save_agent.id)
        await client.agents.delete_agent(booking_agent.id)


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Ansvarsfriskrivning**:
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, vänligen notera att automatiska översättningar kan innehålla fel eller brister. Det ursprungliga dokumentet på dess ursprungsspråk bör betraktas som den auktoritativa källan. För viktig information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för några missförstånd eller felaktiga tolkningar som uppstår vid användning av denna översättning.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
